In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
datapath = o3d.data.DemoICPPointClouds().paths
print(datapath)
source = o3d.io.read_point_cloud(datapath[0])
target = o3d.io.read_point_cloud(datapath[1])

['/home/daiv02/open3d_data/extract/DemoICPPointClouds/cloud_bin_0.pcd', '/home/daiv02/open3d_data/extract/DemoICPPointClouds/cloud_bin_1.pcd', '/home/daiv02/open3d_data/extract/DemoICPPointClouds/cloud_bin_2.pcd']


In [3]:
# test draw
o3d.visualization.draw_geometries([source])

In [4]:
# paint and move pcd
source.paint_uniform_color([0.5, 0.5, 1])
target.paint_uniform_color([1, 0.5, 0.5])
initial_trans = np.identity(4)
initial_trans[0,3] = -3.0

In [5]:
# define draw func
def draw_registration_result(source, target, transformation):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds)

draw_registration_result([source], [target], initial_trans)

In [6]:
# extract features from point clouds
def keypoint_and_feature_extraction(pcd: o3d.geometry.PointCloud, voxel_size):
    keypoints = pcd.voxel_down_sample(voxel_size)
    viewpoint = np.array([0, 0, 0], dtype='float64')
    radius_normal = 2.0 * voxel_size
    keypoints.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    keypoints.orient_normals_towards_camera_location(viewpoint)

    radius_feature = 5.0 * voxel_size
    feature = o3d.pipelines.registration.compute_fpfh_feature(
        keypoints,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

    return keypoints, feature

voxel_size = 0.1
s_kp, s_feature = keypoint_and_feature_extraction(source, voxel_size)
t_kp, t_feature = keypoint_and_feature_extraction(target, voxel_size)

In [7]:
s_kp.paint_uniform_color([0, 1, 0])
t_kp.paint_uniform_color([0, 1, 0])
draw_registration_result([source, s_kp], [target, t_kp], initial_trans)

In [8]:
# ratio test to remove outliers
np_s_feature = s_feature.data.T
np_t_feature = t_feature.data.T

corrs = o3d.utility.Vector2iVector()
threshold = 0.8
for i, feat in enumerate(np_s_feature):
    distance = np.linalg.norm(np_t_feature - feat, axis=1)
    nearest_idx = np.argmin(distance)
    dist_order = np.argsort(distance)
    ratio = distance[dist_order[0]] - distance[dist_order[1]]
    if ratio < threshold:
        corr = np.array([[i], [nearest_idx]], np.int32)
        corrs.append(corr)

print(f'number of correration set: {len(corrs)}')

number of correration set: 1301


In [9]:
# visualize correspondences
def create_lineset_from_correspondenes(corrs_set, pcd1, pcd2, transformation=np.identity(4)):
    pcd1_tmp = copy.deepcopy(pcd1)
    pcd1_tmp.transform(transformation)
    corrs = np.asarray(corrs_set)
    np_points1 = np.array(pcd1_tmp.points)
    np_points2 = np.array(pcd2.points)
    points = list()
    lines = list()

    for i in range(corrs.shape[0]):
        points.append(np_points1[corrs[i, 0]])
        points.append(np_points2[corrs[i, 1]])
        lines.append([2*i, (2*i)+1])

    colors = [np.random.rand(3) for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines =o3d.utility.Vector2iVector(lines)
    )

    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

line_set = create_lineset_from_correspondenes(corrs, s_kp, t_kp, initial_trans)
draw_registration_result([source, s_kp], [target, t_kp, line_set], initial_trans)

In [10]:
# visuzlize those point cloud
trans_ptp = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
trans_all = trans_ptp.compute_transformation(s_kp, t_kp, corrs)
draw_registration_result([source], [target], trans_all)